In [1]:
# default_exp core

# Core

> basic functions

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export
from data_tool.imports import *

In [5]:
#export
def kwargs2str(**kwargs):
    "kwargs to string split with `,`"
    str_list = [f"{k}: {v}" for k, v in kwargs.items()]
    return ", ".join(str_list)

In [17]:
kwargs2str(a='a', b='b', c='ccc')

'a: a, b: b, c: ccc'

In [21]:
#export
def sorted_dict(x, comp='key', **kwargs):
    comp_index = 1 if comp == 'value' else 0
    sorted_x = dict(sorted(x.items(),key=operator.itemgetter(comp_index), **kwargs))
    return sorted_x

In [22]:
test_data = {
    '199': 100,
    '020': 1000,
    '600': 20,
    '333': 200
}
sorted_dict(test_data)

{'020': 1000, '199': 100, '333': 200, '600': 20}

In [23]:
sorted_dict(test_data, comp='value')

{'600': 20, '199': 100, '333': 200, '020': 1000}

In [26]:
sorted_dict(test_data, comp='value', reverse=True)

{'020': 1000, '333': 200, '199': 100, '600': 20}

In [6]:
#export
def get_count_data_datetime(input_df, count_column, date_time_column:str=None, agg_method='sum', cumsum:bool=False, resample_mode:str='d', extra_meta_column:str=None):
    date_df = input_df
    extra_meta_keys = list(date_df[extra_meta_column].value_counts().index) if extra_meta_column is not None else None
    if date_time_column is not None:
        if 'int' in str(date_df[date_time_column].dtype).lower(): date_df[date_time_column] = date_df[date_time_column].astype("string")
        date_df[date_time_column] = pd.to_datetime(date_df[date_time_column])
        date_df = date_df.set_index(date_time_column)
    if date_time_column is None: date_time_column = date_df.index.name = "_index_column_title"
    if resample_mode in ['d', 'w', 'm', 'y']:
        date_df_iter = date_df.resample(resample_mode)
    else:
        date_df_iter = date_df.groupby(date_time_column)
    agg_row = date_df_iter[count_column].agg(agg_method)
    if cumsum: agg_row = agg_row.cumsum()
    count_data = agg_row.to_dict()
    if extra_meta_column is not None:
        extra_output_data = {k: [] for k in extra_meta_keys}
        for o in date_df_iter:
            extra_data_item = o[1].groupby(extra_meta_column)[count_column].sum().to_dict()
            for extra_meta_key in extra_meta_keys:
                extra_output_data[extra_meta_key].append(extra_data_item.get(extra_meta_key, 0))
    else:
        extra_output_data = None
    return (count_data, extra_output_data) if extra_output_data is not None else count_data

In [10]:
df = pd.DataFrame({
 'date': {0: '2000-03-01',
          1: '2000-03-02',
          2: '2000-03-03',
          3: '2000-03-06',
          4: '2000-03-07',
          5: '2000-03-08',
          6: '2000-03-09',
          7: '2000-03-10'},
 'open': {0: 118.56,
          1: 127.0,
          2: 124.87,
          3: 126.0,
          4: 126.44,
          5: 122.87,
          6: 120.87,
          7: 121.69},
 'cat': {0: 'A',
         1: 'B',
         2: 'C',
         3: 'B',
         4: 'A',
         5: 'A',
         6: 'C',
         7: 'C'}
  })